In [1]:
import re
import numpy as np
import pandas as pd 
import missingno as msno
from text_similarity import *

from nltk.corpus import stopwords
portugues_stops = stopwords.words('portuguese')

portugues_stops.append('&')

# mas possivelmente remover com expresões regulares
portugues_stops.append('fio')
portugues_stops.append('cirurgico')

portugues_stops = [i.upper() for i in portugues_stops] 

from sklearn.preprocessing import Binarizer
from sklearn.metrics import confusion_matrix

# Importar os dados

In [2]:
df = pd.read_csv('data/dados-pacote-cirurgico-2022-02-08 10_30_25.csv', sep=';', encoding='iso8859-1')

-  Remover espaços desnecessarios e '*'

In [3]:
size = df['Descrição do Item'].shape[0]

df['Descrição do Item'] = [df['Descrição do Item'][i].replace('*', '').strip() for i in range(size)]
df['Descrição do Item'] = [re.sub(' +',' ', df['Descrição do Item'][i]) for i in range(size)]

- Pegar itens unicos

In [ ]:
descricao_itens = df['Descrição do Item'].unique()

- Remover stopwords

In [ ]:
itens = pd.Series(descricao_itens).apply(lambda item: remove_stowords(item, portugues_stops))

In [ ]:
itens[itens == 'MONONYLON P1662 T 4-0 - JOHNSON JOHNSON']

# Matriz de similaridade

- Gernado a Matriz de similaridade de Jaro

In [ ]:
matriz_jaro = similarity_matrix(np.array(itens), metric = 'jaro_winkler')

- Convertendo matriz de similaridade para Dataframe

In [ ]:
matriz_similaridade = pd.DataFrame(matriz_jaro, index=descricao_itens, columns=descricao_itens) 

In [ ]:
matriz_similaridade

- Converter Matriz para dataframe

In [ ]:
df_similaridade = matriz_similaridade.unstack().reset_index().dropna()

df_similaridade.rename(columns = {'level_0': 'item_A', 'level_1': 'item_B', 0:'valor_similaridade'}, inplace = True)

df_similaridade

## Binarizer

In [ ]:
df_similaridade['similares'] = Binarizer(threshold=0.8).fit_transform(df_similaridade[['valor_similaridade']])

In [ ]:
df_similaridade

In [ ]:
bins = [-1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

labels = [
    "00 -| 10", "11 -| 20", "21 -| 30", "31 -| 40", "41 -| 50", 
    "51 -| 60", "61 -| 70", "71 -| 80", "81 -| 90", "91 -| 100"
]

df_similaridade['classe'] = creat_class(df_similaridade['valor_similaridade'],bins, labels)

# Explorando os Resultados

### FIlrar por valores de similaridade

- Remover valores de 1.0 e 0.0

In [ ]:
df_similaridade = df_similaridade.query('valor_similaridade > 0 and valor_similaridade < 1.0')
df_similaridade.head()

- Pegar similaridade de 90%

In [ ]:
df_similaridade.query('valor_similaridade >= 0.9')

- Pegar similaridade de 80%

In [ ]:
df_similaridade.query('valor_similaridade >= 0.8')

### Analisar por Grupos 

In [ ]:
cateter_cetrofix = [
    'CATETER CERTOFIX DUO PAED S420 G22,22 20CM 4166949',
    'CATETER CERTOFIX MONO PAED S110 G22 10CM 4160177'
]

cateter_venoso = [
    'CATETER VENOSO PERIFERICO 22G - AUTOGUARD 381823 BD',
    'CATETER VENOSO PERIFERICO 24G - AUTOGUARD 381812 BD'
    'CATETER INTRAVASCULAR PERIFERICO JELCO 22GX25MM AZUL - SMITH'
]

coletor = [
    'COLETOR DE URINA INFANTIL SISTEMA FECHADO - PSIMON',
    'COLETOR DE  URINA SISTEMA FECHADO URIFLOW 2000ML - CREMER',
    'COLETOR DE  URINA SISTEMA ABERTO - EMBRAMED',
    'COLETOR DE URINA INFANTIL SISTEMA FECHADO - PSIMON'
]

fio_monolion = [
    'FIO CIRURGICO MONONYLON P1662 T 4-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO MONONYLON P1664 T 4-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO MONONYLON P1663 T 3-0 - JOHNSON E JOHNSON',
    'FIO CIRURGICO MONONYLON 1215 T 2-0 - JOHNSON & JOHNSON'
]

fio_prolene = [
    'FIO CIRURGICO PROLENE 8707 T 6-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 9556 T 5-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE M8703 T 7-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 8704 T 7-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO PROLENE 8730 T 8-0'
]

fio_seda = [
    'FIO CIRURGICO SEDA G813 T 2-0 - JOHNSON & JOHNSON',
    'FIO CIRURGICO SEDA SSP15 T-0'
    'FIO CIRURGICO SEDA SSP13 T 3-0 - JOHNSON & JOHNSON'
    'FIO CIRURGICO SEDA SSP14 T 2-0 - JOHNSON E JOHNSON'
]


fio_vicryl = [
    'FIO CIRURGICO VICRYL VCP316 H - JOHNSON & JOHNSON',
    'FIO CIRURGICO VICRYL VCP340 H 0 - JOHNSON & JOHNSON'
]

bisturi = [
    'LAMINA DE BISTURI NR 11 ESTERIL - B. BRAUN',
    'LAMINA DE BISTURI NR 15 ESTERIL - B. BRAUN',
    'LAMINA DE BISTURI NR 21 ESTERIL - B. BRAUN'
]

perfursor =[
    'PERFUSOR SET 120CM - B. BRAUN',
    'PERFUSOR SET 20CM - B. BRAUN'
]

cauterio = [
    'PLACA DE CAUTERIO INFANTIL SEM CABO REF: 1146 - 3M', 
    'PLACA ELETROCIRURGICA PARA PACIENTE AUTOADESIVA RN'
]

frauda = [
    'FRALDA NEONATAL TAM. PP (0 A 3,5KG)',
    'FRALDA INFANTIL P (ATE 6KG)',
    'FRALDA INFANTIL EG (ACIMA DE 12KG)',
    'FRALDA INFANTIL M (DE 6 A 10KG)',
    'FRALDA PROTEK XXG 14 A 18 KG'
]

sonda = [
    'SONDA FOLEY 2 VIAS 06 SILICONE - RUSCH',
    'SONDA FOLEY 2 VIAS 08 BALAO 30 - RUSCH',
    'SONDA FOLEY 2 VIAS 08 SILICONE - RUSCH'
]

tubo_endotraqueal = [
    'TUBO ENDOTRAQUEAL S/BALAO 3.0 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL S/BALAO 4.0 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL S/BALAO 3.5 - MALLINCKRODT',
    'TUBO ENDOTRAQUEAL C/BALAO 4.0 - MALLINCKRODT'
]

sistema_drenagem = [
    'SISTEMA DRENAGEM MEDIASTINAL Nº 10 250ML',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 12 250ML',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 16 2000ML P.SIMON',
    'SISTEMA DRENAGEM MEDIASTINAL Nº 18 2000ML P.SIMON'
]

- Filtro cateter cetrofix

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(cateter_cetrofix)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(cateter_cetrofix)].query('valor_similaridade >= 0.8')

- Filtro cateter venoso

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(cateter_venoso)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(['CATETER INTRAVASCULAR PERIFERICO JELCO 22GX25MM AZUL - SMITH'])].query('similares != 0')

- Filtro Coletor

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(coletor)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(coletor)].query('valor_similaridade >= 0.8')

- Filtro fio Prolene

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_prolene)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_prolene)].query('valor_similaridade >= 0.8 and valor_similaridade <= 0.9')

- Filtro Fio seda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_seda)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_seda)].query('valor_similaridade >= 0.8 and valor_similaridade <= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(['FIO CIRURGICO SEDA SSP15 T-0'])].query('similares != 0')

In [ ]:
df_fio_seda_08 == 'FIO CIRURGICO SEDA SSP15 T-0'

- Filtro Fio Vicryl

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_vicryl)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_vicryl)].query('valor_similaridade >= 0.8')

- Filtro bisturi

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(bisturi)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(bisturi)].query('valor_similaridade >= 0.8')

- Filtro Perfursor

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(perfursor)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(perfursor)].query('valor_similaridade >= 0.8')

- Fltro Cauterio

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(cauterio)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(cauterio)].query('valor_similaridade >= 0.8')

- Filtro tubo endotraqueal

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(tubo_endotraqueal)].query('valor_similaridade >= 0.9')


In [ ]:
df_similaridade[df_similaridade['item_B'].isin(tubo_endotraqueal)].query('valor_similaridade >= 0.8')

- Filtrando fio monolion

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_monolion)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(fio_monolion)].query('valor_similaridade >= 0.8')

- Filtrando frauda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(frauda)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(frauda)].query('valor_similaridade >= 0.8')

- Filtro Sonda

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(sonda)].query('valor_similaridade >= 0.9')

In [ ]:
df_similaridade[df_similaridade['item_B'].isin(sonda)].query('valor_similaridade >= 0.8')
